In [25]:
""""
Experiment to check how a Random Forest Classifier reacts against
modifications in the synthetic testing datasets.
"""

import torch
from data import load_csv_to_df, train_test_split_stratify
from graph import get_flow_features_values, get_encoded_label
import configparser
from sklearn.ensemble import RandomForestClassifier
from data import modify_portscan_attack_behavior
import os
import metrics
#!pip install plotly

# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint






In [30]:

source_code_dir = '/home/dev/src'

print(os.getcwd())

config = configparser.ConfigParser()
config.read(source_code_dir+"/config.ini")


data = load_csv_to_df(
    config["PARAMETERS"]["DataFolderPath"] + config["PARAMETERS"]["TrainFile"])

# test = load_csv_to_df(
#     config["PARAMETERS"]["DataFolderPath"] + config["PARAMETERS"]["EvalFile"])

# 데이터 셋 분리 
train,test  = train_test_split_stratify(data,0.3)

print('train_data label rate',train[' Label'].value_counts())
print()
print('test_data label rate',test[' Label'].value_counts())


/home/dev/src
train_data label rate  Label
PortScan    101715
BENIGN       81623
Name: count, dtype: int64

test_data label rate  Label
PortScan    25429
BENIGN      20406
Name: count, dtype: int64


In [27]:

labels_map = {"BENIGN": 0, "PortScan": 1}
labels = ["BENIGN", "MALIGN"]

flow_features = [    
    " Flow Duration",
    " Flow Packets/s",
    " Flow IAT Mean",
    " Fwd IAT Mean",
    " Bwd IAT Mean",]

# flow_features = [
#     " Average Packet Size",
#     " Flow IAT Mean",
#     " Flow Duration",
# ]


test = modify_portscan_attack_behavior(test)


In [28]:
# train vectors
x_train = []
y_train = []
for _, row in train.iterrows():
    x_train.append(get_flow_features_values(row, flow_features))
    y_train.append(get_encoded_label(
        row, labels_map))

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

# test vectors
x_test = []
y_test = []
for _, row in test.iterrows():
    x_test.append(get_flow_features_values(row, flow_features))
    y_test.append(get_encoded_label(
        row, labels_map))

x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)



In [51]:
y_train_dummie = np.array(pd.get_dummies(y_train,dtype=int))
y_test_dummie = np.array(pd.get_dummies(y_test,dtype=int))

x_test


tensor([[4.6600e+02, 4.2918e+03, 4.6600e+02, 0.0000e+00, 0.0000e+00],
        [3.8000e+01, 5.2632e+04, 3.8000e+01, 0.0000e+00, 0.0000e+00],
        [1.1000e+01, 1.8182e+05, 1.1000e+01, 0.0000e+00, 0.0000e+00],
        ...,
        [7.4000e+01, 2.7027e+04, 7.4000e+01, 0.0000e+00, 0.0000e+00],
        [6.7000e+01, 2.9851e+04, 6.7000e+01, 0.0000e+00, 0.0000e+00],
        [7.2000e+01, 2.7778e+04, 7.2000e+01, 0.0000e+00, 0.0000e+00]])

In [52]:
def model():
    model = Sequential()
    model.add(Dense(64,activation='relu', input_shape=(5,1)))

    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [54]:
model = model()
model.summary()

checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

history = model.fit(x_train, y_train_dummie, epochs=20, batch_size=128, validation_data=(x_test, y_test_dummie), callbacks=[checkpoint])

ValueError: The first argument to `Layer.call` must always be passed.

In [12]:
print("Training model...")
model.fit(x_train, y_train)
print("Evaluating model...")

print(model.score(x_test, y_test))


pred = model.predict(x_test)
metrics.compute_metrics(y_test, pred)

{'F1 Score': 0.962571317519216,
 'Accuracy': 0.9633249700010909,
 'Recall': 0.9592951347120982,
 'Precision': 0.9679822953069751}